In [70]:
import pandas as pd
import numpy as np
from requests import requests
from src.libs.utils.functions import *

In [71]:
countries_req=requests.get('https://api.mercadolibre.com/sites')
countries=countries_req.json()
country_selected=filter_list_of_dicts_by_string_in_key(list_of_dicts=countries,key_name='name',string_value='Col')
print(country_selected)

[{'default_currency_id': 'COP', 'id': 'MCO', 'name': 'Colombia'}]


In [72]:
country_id=country_selected[0].get('id')
cats = requests.get(f'https://api.mercadolibre.com/sites/{country_id}/categories')
categorias=cats.json()
cat_selected=filter_list_of_dicts_by_string_in_key(list_of_dicts=categorias,key_name='name',string_value='Cel')
print(cat_selected)

[{'id': 'MCO1051', 'name': 'Celulares y Teléfonos'}]


In [73]:
cat_id = cat_selected[0].get('id')
offset = range(0, 951, 50)
df_items=pd.DataFrame()
for i in offset:
    url = f'https://api.mercadolibre.com/sites/MCO/search?category={cat_id}&offset={i}'
    itms = requests.get(url)
    items_tmp=itms.json()
    df_items=pd.concat([df_items,pd.DataFrame(items_tmp.get('results'))],axis=0,ignore_index=True)

In [74]:
df_items.columns

Index(['id', 'title', 'condition', 'thumbnail_id', 'catalog_product_id',
       'listing_type_id', 'permalink', 'buying_mode', 'site_id', 'category_id',
       'domain_id', 'thumbnail', 'currency_id', 'order_backend', 'price',
       'original_price', 'sale_price', 'sold_quantity', 'available_quantity',
       'official_store_id', 'use_thumbnail_id', 'accepts_mercadopago', 'tags',
       'shipping', 'stop_time', 'seller', 'seller_address', 'address',
       'attributes', 'installments', 'winner_item_id', 'catalog_listing',
       'discounts', 'promotions', 'inventory_id', 'official_store_name',
       'differential_pricing', 'variation_filters', 'variations_data'],
      dtype='object')

In [75]:
df_items=expand_dict_column(df_items,'seller')

In [63]:
[c for c in df_items.columns if 'price' in c]

['price', 'original_price', 'sale_price']

In [64]:
[c for c in df_items.columns if 'seller' in c]

['seller_address',
 'seller_reputation.level_id',
 'seller_reputation.power_seller_status',
 'seller_reputation.transactions.canceled',
 'seller_reputation.transactions.completed',
 'seller_reputation.transactions.period',
 'seller_reputation.transactions.ratings.negative',
 'seller_reputation.transactions.ratings.neutral',
 'seller_reputation.transactions.ratings.positive',
 'seller_reputation.transactions.total',
 'seller_reputation.metrics.sales.period',
 'seller_reputation.metrics.sales.completed',
 'seller_reputation.metrics.claims.period',
 'seller_reputation.metrics.claims.rate',
 'seller_reputation.metrics.claims.value',
 'seller_reputation.metrics.delayed_handling_time.period',
 'seller_reputation.metrics.delayed_handling_time.rate',
 'seller_reputation.metrics.delayed_handling_time.value',
 'seller_reputation.metrics.cancellations.period',
 'seller_reputation.metrics.cancellations.rate',
 'seller_reputation.metrics.cancellations.value',
 'seller_reputation.metrics.claims.excl

In [37]:
sellers_cat=np.unique(get_list_values_by_key_of_dict([c for c in df_items.seller],key_name='id'))
df_reputation=pd.DataFrame()
for i in sellers_cat:
    url = f'https://api.mercadolibre.com/users/{i}'
    rep = requests.get(url)
    reputation_tmp = rep.json()
    if isinstance(reputation_tmp, dict):
        df_reputation = pd.concat([df_reputation, pd.DataFrame([reputation_tmp])], axis=0)
    elif isinstance(reputation_tmp, list):
        df_reputation = pd.concat([df_reputation, pd.DataFrame(reputation_tmp)], axis=0)
df_reputation.reset_index(drop=True,inplace=True)        

In [38]:
df_reputation.head()

,id,nickname,registration_date,country_id,address,user_type,tags,logo,points,site_id,permalink,seller_reputation,buyer_reputation,status
0,531347,TRAYGO,2000-08-14T00:00:00.000-04:00,CO,"{'city': 'Suba', 'state': 'CO-DC'}",normal,"[normal, eshop, mshops]",None,3551,MCO,http://perfil.mercadolibre.com.co/TRAYGO,"{'level_id': '5_green', 'power_seller_status':...",{'tags': []},{'site_status': 'active'}
1,1841943,PRODUCTSTORE,2006-11-16T21:46:52.000-04:00,CO,"{'city': 'Engativá', 'state': 'CO-DC'}",normal,"[normal, mshops]",None,128,MCO,http://perfil.mercadolibre.com.co/PRODUCTSTORE,"{'level_id': '5_green', 'power_seller_status':...",{'tags': []},{'site_status': 'active'}
2,18998983,SERGIOJIME,2007-05-27T15:28:36.000-04:00,CO,"{'city': 'Cartagena De Indias', 'state': 'CO-B...",normal,"[normal, eshop, mshops]",None,392,MCO,http://perfil.mercadolibre.com.co/SERGIOJIME,"{'level_id': '5_green', 'power_seller_status':...",{'tags': []},{'site_status': 'active'}
3,21932236,SOFT PAWS,2010-01-06T05:54:47.000-04:00,CO,"{'city': 'Usaquén', 'state': 'CO-DC'}",normal,"[normal, eshop, mshops]",None,1017,MCO,http://perfil.mercadolibre.com.co/SOFT+PAWS,"{'level_id': '4_light_green', 'power_seller_st...",{'tags': []},{'site_status': 'active'}
4,22672010,TIENDACHICOLISTO,2010-01-20T12:39:06.000-04:00,CO,"{'city': 'Teusaquillo', 'state': 'CO-DC'}",normal,"[normal, mshops]",None,1413,MCO,http://perfil.mercadolibre.com.co/TIENDACHICOL...,"{'level_id': '5_green', 'power_seller_status':...",{'tags': []},{'site_status': 'active'}


In [39]:
df_reputation.columns

Index(['id', 'nickname', 'registration_date', 'country_id', 'address',
       'user_type', 'tags', 'logo', 'points', 'site_id', 'permalink',
       'seller_reputation', 'buyer_reputation', 'status'],
      dtype='object')

In [40]:
df_reputation=(
    df_reputation
    .reindex(columns=['id', 'nickname','registration_date','tags','points','seller_reputation','status'])
)

In [41]:
df_reputation=expand_dict_column(df_reputation,'seller_reputation')
df_reputation=expand_dict_column(df_reputation,'status')

In [42]:
df_reputation.head()

,id,nickname,registration_date,tags,points,level_id,power_seller_status,transactions.canceled,transactions.completed,transactions.period,transactions.ratings.negative,transactions.ratings.neutral,transactions.ratings.positive,transactions.total,real_level,protection_end_date,site_status
0,531347,TRAYGO,2000-08-14T00:00:00.000-04:00,"[normal, eshop, mshops]",3551,5_green,platinum,232,9557,historic,0.01,0.01,0.98,9789,NaN,NaN,active
1,1841943,PRODUCTSTORE,2006-11-16T21:46:52.000-04:00,"[normal, mshops]",128,5_green,silver,47,2193,historic,0.03,0.00,0.97,2240,NaN,NaN,active
2,18998983,SERGIOJIME,2007-05-27T15:28:36.000-04:00,"[normal, eshop, mshops]",392,5_green,platinum,844,14332,historic,0.09,0.02,0.89,15176,NaN,NaN,active
3,21932236,SOFT PAWS,2010-01-06T05:54:47.000-04:00,"[normal, eshop, mshops]",1017,4_light_green,None,28,490,historic,0.03,0.01,0.96,518,yellow,2023-09-09T12:26:43.000-04:00,active
4,22672010,TIENDACHICOLISTO,2010-01-20T12:39:06.000-04:00,"[normal, mshops]",1413,5_green,platinum,231,6756,historic,0.05,0.03,0.92,6987,NaN,NaN,active
